In [1]:
import pandas as pd
import numpy as np

from capstone_functions import get_admission

triage = pd.read_csv('DATA/triage.csv')
edstays = pd.read_csv('data/edstays.csv')

triage.head()

ModuleNotFoundError: No module named 'capstone_functions'

In [10]:
triage.chiefcomplaint = [str(x).lower() for x in triage.chiefcomplaint]

In [19]:
triage.chiefcomplaint.value_counts()

abd pain                               16601
chest pain                             14619
s/p fall                                7902
dyspnea                                 7758
headache                                5377
                                       ...  
confusion, difficulty finding words        1
weakness, low k                            1
l eye redness/pain                         1
back pain   s/p fall                       1
car vs. house                              1
Name: chiefcomplaint, Length: 61228, dtype: int64

In [21]:
edstays.head()

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition
0,10000032,22595853.0,33258284,6/5/2180 19:17,6/5/2180 23:30,F,WHITE,AMBULANCE,ADMITTED
1,10000032,22841357.0,38112554,26/6/2180 15:54,26/6/2180 21:31,F,WHITE,AMBULANCE,ADMITTED
2,10000032,25742920.0,35968195,5/8/2180 20:58,6/8/2180 1:44,F,WHITE,AMBULANCE,ADMITTED
3,10000032,29079034.0,32952584,22/7/2180 16:24,23/7/2180 5:54,F,WHITE,AMBULANCE,HOME
4,10000032,29079034.0,39399961,23/7/2180 5:54,23/7/2180 14:00,F,WHITE,AMBULANCE,ADMITTED


In [23]:
edstays_sub = edstays[edstays.disposition.isin(['ADMITTED', 'HOME','TRANSFER'])]
edstays_sub.disposition.value_counts()

HOME        254545
ADMITTED    166303
TRANSFER      7436
Name: disposition, dtype: int64

In [25]:
edstays_sub['admitted'] = edstays_sub.apply(get_admission,axis=1) 
edstays_sub.admitted.value_counts()

c:\users\samgh\py_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0    254545
1    173739
Name: admitted, dtype: int64

In [26]:
# what tokens in complaints are most common in each

all_data = pd.merge(edstays_sub, triage, on=['subject_id','stay_id'], how='left')

In [31]:
all_data[['admitted']].value_counts()

admitted
0           254545
1           173739
dtype: int64

In [34]:
all_data[['admitted', 'chiefcomplaint']].value_counts()

admitted  chiefcomplaint                              
0         chest pain                                      9106
          abd pain                                        8461
1         abd pain                                        7266
          dyspnea                                         5406
0         s/p fall                                        4765
                                                          ... 
1         back pain, positive blood cultures, transfer       1
          back pain, pneumonia, transfer                     1
          back pain, pneumonia                               1
          back pain, pleuritic chest pain                    1
0          10 min vision change                              1
Length: 67962, dtype: int64

In [98]:
home_tokens = all_data[all_data.admitted==0]['chiefcomplaint']
home_tokens = [x.split(', ') for x in home_tokens]
home_tokens_flat = list()
for tokens in home_tokens:
    for token in tokens:
        home_tokens_flat.append(token)
home_counts = pd.DataFrame(data={'home_count':home_tokens_flat}).home_count.value_counts()

home_counts_df = pd.DataFrame(home_counts).reset_index()


admit_tokens = all_data[all_data.admitted==1]['chiefcomplaint']
admit_tokens = [x.split(', ') for x in admit_tokens]
admit_tokens_flat = list()
for tokens in admit_tokens:
    for token in tokens:
        admit_tokens_flat.append(token)
admit_counts = pd.DataFrame(data={'admit_count':admit_tokens_flat}).admit_count.value_counts()

admit_counts_df = pd.DataFrame(admit_counts).reset_index()

all_tokens = all_data['chiefcomplaint']
all_tokens = [x.split(', ') for x in all_tokens]
all_tokens_flat = list()
for tokens in all_tokens:
    for token in tokens:
        all_tokens_flat.append(token)
total_counts = pd.DataFrame(data={'total':all_tokens_flat}).total.value_counts()

total_counts_df = pd.DataFrame(total_counts).reset_index()

all_counts = pd.merge(
    pd.merge(total_counts_df, home_counts_df, on='index', how='left'),
    admit_counts_df, on = 'index', how='left')
all_counts

all_counts = all_counts.fillna(0)

all_counts['home_pp'] = all_counts['home_count']/all_counts['total']
all_counts['admit_pp'] = all_counts['admit_count']/all_counts['total']


In [110]:
all_counts_sub = all_counts[all_counts.total > 10]

In [106]:
all_counts['home_pp'] = all_counts['home_count']/all_counts['total']
all_counts['admit_pp'] = all_counts['admit_count']/all_counts['total']


In [112]:
all_counts_sub.sort_values(by='home_pp')

,index,total,home_count,admit_count,home_pp,admit_pp
1321,cold foot,12,0.0,12.0,0.0,1.0
874,pelvic fx,22,0.0,22.0,0.0,1.0
1196,for ercp,13,0.0,13.0,0.0,1.0
1385,dissection,11,0.0,11.0,0.0,1.0
1179,tib/fib fx,14,0.0,14.0,0.0,1.0
...,...,...,...,...,...,...
1211,sinus congestion,13,13.0,0.0,1.0,0.0
1214,accidental ingestion,13,13.0,0.0,1.0,0.0
727,chin lac,32,32.0,0.0,1.0,0.0
1192,rabies shot,13,13.0,0.0,1.0,0.0


In [113]:
all_counts_sub.to_csv('chiefcomplaint_token_counts_by_admit.csv')

In [114]:
# look at exceptions with high (rest of triage data)

In [118]:
edstays.disposition.value_counts()

HOME                           254545
ADMITTED                       166303
TRANSFER                         7436
LEFT WITHOUT BEING SEEN          6516
ELOPED                           6011
OTHER                            4520
LEFT AGAINST MEDICAL ADVICE      1986
EXPIRED                           395
Name: disposition, dtype: int64

In [135]:
expired_df = edstays[edstays.disposition.isin(['ADMITTED','TRANSFER','EXPIRED'])]

def get_expired(x):
    if x['disposition'] == 'EXPIRED':
        return 1
    else:
        return 0

expired_df['expired'] = expired_df.apply(get_expired,axis=1) 

c:\users\samgh\py_env\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [136]:
expired_df.expired.value_counts()

0    173739
1       395
Name: expired, dtype: int64

In [139]:
all_data = pd.merge(expired_df, triage, on=['subject_id','stay_id'], how='left')
pd.crosstab(all_data.gender, all_data.expired)

expired,0,1
gender,,
F,89156,183
M,84583,212


In [141]:
home_tokens = all_data[all_data.expired==0]['chiefcomplaint']
home_tokens = [x.split(', ') for x in home_tokens]
home_tokens_flat = list()
for tokens in home_tokens:
    for token in tokens:
        home_tokens_flat.append(token)
home_counts = pd.DataFrame(data={'discharge_count':home_tokens_flat}).discharge_count.value_counts()

home_counts_df = pd.DataFrame(home_counts).reset_index()


admit_tokens = all_data[all_data.expired==1]['chiefcomplaint']
admit_tokens = [x.split(', ') for x in admit_tokens]
admit_tokens_flat = list()
for tokens in admit_tokens:
    for token in tokens:
        admit_tokens_flat.append(token)
admit_counts = pd.DataFrame(data={'expire_count':admit_tokens_flat}).expire_count.value_counts()

admit_counts_df = pd.DataFrame(admit_counts).reset_index()

all_tokens = all_data['chiefcomplaint']
all_tokens = [x.split(', ') for x in all_tokens]
all_tokens_flat = list()
for tokens in all_tokens:
    for token in tokens:
        all_tokens_flat.append(token)
total_counts = pd.DataFrame(data={'total':all_tokens_flat}).total.value_counts()

total_counts_df = pd.DataFrame(total_counts).reset_index()

all_counts = pd.merge(
    pd.merge(total_counts_df, home_counts_df, on='index', how='left'),
    admit_counts_df, on = 'index', how='left')
all_counts

all_counts = all_counts.fillna(0)

all_counts['discharge_pp'] = all_counts['discharge_count']/all_counts['total']
all_counts['expired_pp'] = all_counts['expire_count']/all_counts['total']

In [144]:
all_counts.to_csv('expire_counts_complaints.csv')